In [0]:
%sql
create volume dev.bronze.landing
comment "This is landing managed volume"

In [0]:
dbutils.fs.mkdirs('dbfs:/databricks-datasets/definitive-guide/data/retail-data/by-day/2010-12-01.csv')

In [0]:
dbutils.fs.cp("/databricks-datasets/definitive-guide/data/retail-data/by-day/2010-12-01.csv","/Volumes/dev/bronze/landing/input")

In [0]:
dbutils.fs.cp("/databricks-datasets/definitive-guide/data/retail-data/by-day/2010-12-02.csv","/Volumes/dev/bronze/landing/input")

In [0]:
%sql
-- create place holder table dev.bronze.invoice_cp(Placeholder table doesn't have any schema, so no columns ill be there)

create table dev.bronze.invoice_cp;

In [0]:
%sql
--Use COPY NTO Command to load into place holder table
COPY INTO dev.bronze.invoice_cp 
from "/Volumes/dev/bronze/landing/input" 
FILEFORMAT = CSV
PATTERN = "*.csv"
Format_options (
  "mergeSchema" = "true",
  "header" = "true"
)
COPY_OPTIONS (
  "mergeSchema" = "true"
);

In [0]:
%sql

select * from dev.bronze.invoice_cp

In [0]:
%sql
describe extended dev.bronze.invoice_cp

In [0]:
%sql

--create new table with only 3 columns invoice_cp_alt and custom columns_insert_date
create table dev.bronze.invoice_cp_alt
(
  InvoiceNo string,
  StockCode string,
  Quantity double,
  _inser_date timestamp
) 

In [0]:
%sql
COPY INTO dev.bronze.invoice_cp_alt 
from (
    select InvoiceNo, StockCode, cast(quantity  as double) Quantity, current_timestamp( ) _inser_date
    from
    "/Volumes/dev/bronze/landing/input"
    ) 
FILEFORMAT = CSV
PATTERN = "*.csv"
Format_options (
  "mergeSchema" = "true",
  "header" = "true"
)
;

In [0]:
%sql
select * from dev.bronze.invoice_cp_alt

In [0]:

## Check whether the Copy INTO command is working or not incrementaly, I will load the data 03 and will check.

dbutils.fs.cp("/databricks-datasets/definitive-guide/data/retail-data/by-day/2010-12-03.csv","/Volumes/dev/bronze/landing/input")

In [0]:
%sql
COPY INTO dev.bronze.invoice_cp_alt 
from (
    select InvoiceNo, StockCode, cast(quantity  as double) Quantity, current_timestamp( ) _inser_date
    from
    "/Volumes/dev/bronze/landing/input"
    ) 
FILEFORMAT = CSV
PATTERN = "*.csv"
Format_options (
  "mergeSchema" = "true",
  "header" = "true"
)
;

In [0]:
%sql
select count(1) from dev.bronze.invoice_cp_alt